## Connect Google Drive and GPU


In [ ]:
%reset

# connect google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# connect colab gpu
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

## Import Needed Libraries, Paramaters and Functions

In [2]:
import sys
import time
import os.path
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils import prune
import torchvision
import matplotlib.pyplot as plt
from torch.utils.mobile_optimizer import optimize_for_mobile
from scipy import stats
from sklearn.utils import shuffle

SEED = 10
WINDOW_SIZE = 128
FEATURE_SIZE = 9
LABEL_SIZE = 6
BATCH_SIZE = 32
PATH = '/content/drive/MyDrive/CNNPaper'
TRAIN_LOADER_PATH = PATH + '/model/final/loader/train_loader'
VALID_LOAER_PATH = PATH + '/model/final/loader/valid_loader'
TEST_LOADER_PATH = PATH + '/model/final/loader/test_loader'
TRAIN_DATA_DIR_PATH = PATH + '/data/UCIHAR/train/'
TEST_DATA_DIR_PATH = PATH + '/data/UCIHAR/test/'
TRAIN_DATA_PATH = PATH + '/data/UCIHAR/train_data.cvs'
TEST_DATA_PATH = PATH + '/data/UCIHAR/test_data.cvs'

In [3]:
def read_data(file_path, TYPE):
  """
    Read data from file_path
    Paramater:
      file_path: str
    Return:
      a DataFrame with the data and labels 
  """
  if os.path.isfile(TRAIN_DATA_PATH) and TYPE == 'train':
    print("Start reading data ...")
    df = pd.read_csv(TRAIN_DATA_PATH)
    print("Finish reading data ...")
  elif os.path.isfile(TEST_DATA_PATH) and TYPE == 'test':
    print("Start reading data ...")
    df = pd.read_csv(TEST_DATA_PATH)
    print("Finish reading data ...")
  else:
    print("Start reading data ...")
    x_accel = pd.read_csv(file_path + 'Inertial Signals/body_acc_x_'+TYPE+'.txt', header=None, names=['x-accel'])
    y_accel = pd.read_csv(file_path + 'Inertial Signals/body_acc_y_'+TYPE+'.txt', header=None, names=['y-accel'])
    z_accel = pd.read_csv(file_path + 'Inertial Signals/body_acc_z_'+TYPE+'.txt', header=None, names=['z-accel'])
    x_gyro = pd.read_csv(file_path + 'Inertial Signals/body_gyro_x_'+TYPE+'.txt', header=None, names=['x-gyro'])
    y_gyro = pd.read_csv(file_path + 'Inertial Signals/body_gyro_y_'+TYPE+'.txt', header=None, names=['y-gyro'])
    z_gyro = pd.read_csv(file_path + 'Inertial Signals/body_gyro_z_'+TYPE+'.txt', header=None, names=['z-gyro'])
    x_gyro_total = pd.read_csv(file_path + 'Inertial Signals/total_acc_x_'+TYPE+'.txt', header=None, names=['total-x-gyro'])
    y_gyro_total = pd.read_csv(file_path + 'Inertial Signals/total_acc_y_'+TYPE+'.txt', header=None, names=['total-y-gyro'])
    z_gyro_total = pd.read_csv(file_path + 'Inertial Signals/total_acc_z_'+TYPE+'.txt', header=None, names=['total-z-gyro'])
    activity = pd.read_csv(file_path + 'y_'+TYPE+'.txt', header=None, names=['activity'])
    length = len(x_accel)
    df = pd.DataFrame(columns=['user', 'activity', 'timestamp', 'x-accel', 'y-accel', 'z-accel', 'total-x-gyro', 'total-y-gyro', 'total-z-gyro'])
    
    for i in range(length):
      x_acc_data = x_accel['x-accel'][i].split()
      y_acc_data = y_accel['y-accel'][i].split()
      z_acc_data = z_accel['z-accel'][i].split()
      x_gyro_data = x_gyro['x-gyro'][i].split()
      y_gyro_data = y_gyro['y-gyro'][i].split()
      z_gyro_data = z_gyro['z-gyro'][i].split()
      total_gyro_x_data = x_gyro_total['total-x-gyro'][i].split()
      total_gyro_y_data = y_gyro_total['total-y-gyro'][i].split()
      total_gyro_z_data = z_gyro_total['total-z-gyro'][i].split()
      activity_data = activity['activity'][i]
      print("index " + str(i))
      size = len(x_acc_data)
      for j in range(size):
        df = df.append({'user': i, 'activity': activity_data, 'timestamp': j, 
                                'x-accel': float(x_acc_data[j]),
                                'y-accel': float(y_acc_data[j]),
                                'z-accel': float(z_acc_data[j]),
                                'x-gyro': float(x_gyro_data[j]),
                                'y-gyro': float(y_gyro_data[j]),
                                'z-gyro': float(z_gyro_data[j]),
                                'total-x-gyro': float(total_gyro_x_data[j]),
                                'total-y-gyro': float(total_gyro_y_data[j]),
                                'total-z-gyro': float(total_gyro_z_data[j])}, ignore_index=True)
    print("Finish reading data ...")
    if TYPE == 'train':
      df.to_csv('train_data.cvs', index=False)
    else:
      df.to_csv('test_data.cvs', index=False)
  return df

def feature_normalize(data):
  """
    Normalize the feature data
    Paramater:
      data: a list of floats
    Return:
      a list of floats with normalized data
  """
  mu = np.mean(data, axis=0)
  sigma = np.std(data, axis=0)
  return (data - mu) / sigma

def dataset_normalize(dataset):
  """
    Normalize the whole dataset
    Paramater:
      dataset: a DataFrame with the data and labels 
    Return:
      a DataFrame with the normalized data and labels 
  """
  dataset.dropna(axis=0, how='any', inplace=True)
  print("Normalizing x-accel ...")
  dataset['x-accel'] = feature_normalize(dataset['x-accel'])
  print("Normalizing y-accel ...")
  dataset['y-accel'] = feature_normalize(dataset['y-accel'])
  print("Normalizing z-accel ...")
  dataset['z-accel'] = feature_normalize(dataset['z-accel'])
  print("Normalizing x-gyro ...")
  dataset['x-gyro'] = feature_normalize(dataset['x-gyro'])
  print("Normalizing y-gyro ...")
  dataset['y-gyro'] = feature_normalize(dataset['y-gyro'])
  print("Normalizing z-gyro ...")
  dataset['z-gyro'] = feature_normalize(dataset['z-gyro'])
  print("Normalizing total-x-gyro ...")
  dataset['total-x-gyro'] = feature_normalize(dataset['total-x-gyro'])
  print("Normalizing total-y-gyro ...")
  dataset['total-x-gyro'] = feature_normalize(dataset['total-x-gyro'])
  print("Normalizing total-z-gyro ...")
  dataset['total-x-gyro'] = feature_normalize(dataset['total-x-gyro'])
  return dataset

def dataset_segmentation(data):
  """
    Dataset segmentation according the window size
    Paramater:
      data: a list of floats
    Return:
      segments and labels 
  """
  print("Start segmentation with window size: ", WINDOW_SIZE)
  segments = np.empty((0, WINDOW_SIZE, FEATURE_SIZE))
  labels = np.empty((0))
  size = data['timestamp'].count()
  for start in range(0, size, WINDOW_SIZE):
      x1 = data["x-accel"][start:start+WINDOW_SIZE]
      y1 = data["y-accel"][start:start+WINDOW_SIZE]
      z1 = data["z-accel"][start:start+WINDOW_SIZE]
      x2 = data["x-gyro"][start:start+WINDOW_SIZE]
      y2 = data["y-gyro"][start:start+WINDOW_SIZE]
      z2 = data["z-gyro"][start:start+WINDOW_SIZE]
      x3 = data["total-x-gyro"][start:start+WINDOW_SIZE]
      y3 = data["total-y-gyro"][start:start+WINDOW_SIZE]
      z3 = data["total-z-gyro"][start:start+WINDOW_SIZE]
      if len(data["timestamp"][start:start+WINDOW_SIZE]) == WINDOW_SIZE:
        segments = np.vstack([segments, np.dstack([x1,y1,z1,x2,y2,z2,x3,y3,z3])])
        labels = np.append(labels, stats.mode(data["activity"][start:start+WINDOW_SIZE])[0][0])
  labels = np.asarray(pd.get_dummies(labels), dtype = np.int8)
  segments = segments.reshape(len(segments), FEATURE_SIZE, WINDOW_SIZE)
  print("Finish segmentation ...")
  return segments, labels

def train_valid_test_split(segments, classes, test_x, test_y, k_fold):
  """
    Split train, valid and test datase
    Paramater:
      segments: a list of input data
      classes: a list of classes data
      k: k fold cross validation
    Return:
      segments and labels 
  """
  print("Start dataset split... ")
  seg_len = len(segments)
  idx_val = [0, int(seg_len/5*1), int(seg_len/5*2), int(seg_len/5*3), int(seg_len/5*4), seg_len]
  train_range1 = range(0, idx_val[k_fold])
  valid_range = range(idx_val[k_fold], idx_val[k_fold+1])
  train_range2 = range(idx_val[k_fold+1], seg_len)

  train_x = np.concatenate((segments[train_range1], segments[train_range2]), axis=0)
  train_y = np.concatenate((classes[train_range1], classes[train_range2]), axis=0)
  valid_x = segments[valid_range]
  valid_y = classes[valid_range]

  # get train data
  train_data = []
  for i in range(len(train_x)):
    train_data.append([train_x[i], train_y[i]])
  
  # get valid data
  valid_data = []
  for i in range(len(valid_x)):
    valid_data.append([valid_x[i], valid_y[i]])
  
  # get test data
  test_data = []
  for i in range(len(test_x)):
    test_data.append([test_x[i], test_y[i]])
  print(len(train_data))
  print(len(valid_data))
  print(len(test_data))

  # generate DataLoader for each dataset
  trainloader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=BATCH_SIZE)
  validloader = torch.utils.data.DataLoader(valid_data, shuffle=True, batch_size=BATCH_SIZE)
  testloader = torch.utils.data.DataLoader(test_data, shuffle=True, batch_size=BATCH_SIZE)
  
  print("Finish dataset split... ")
  return trainloader, validloader, testloader


## Load and Save Train, Test, Valid Dataset

In [ ]:
train_dataset = dataset_normalize(read_data(TRAIN_DATA_DIR_PATH, 'train'))
test_dataset = dataset_normalize(read_data(TEST_DATA_DIR_PATH, 'test'))
segments, classes = dataset_segmentation(train_dataset)
test_x, test_y = dataset_segmentation(test_dataset)
np.random.seed(SEED)
total_x, total_y = shuffle(segments, classes)

In [ ]:
cross_valid_range = 5

for k in range(cross_valid_range):
  print("Start spliting for k = " + str(k))
  trainloader, validloader, testloader = train_valid_test_split(total_x, total_y, test_x, test_y, k)
  CROSS_TRAIN_LOADER_PATH = TRAIN_LOADER_PATH + str(k) + '.pkl'
  CROSS_VALID_LOADER_PATH = VALID_LOAER_PATH + str(k) + '.pkl'
  CROSS_TEST_LOADER_PATH = TEST_LOADER_PATH + str(k) + '.pkl'
  torch.save(trainloader, CROSS_TRAIN_LOADER_PATH)
  torch.save(validloader, CROSS_VALID_LOADER_PATH)
  torch.save(testloader, CROSS_TEST_LOADER_PATH)
  print("Finish data loading...")

## Load CNN Model and Other Helper Functions


In [4]:
# Here are the variables that you can modify
NODE_SIZE = 128
KERNAL_SIZE = 5
LEARNING_RATE = 0.0001
k = 4 # cross validation fold

In [5]:
CROSS_TRAIN_LOADER_PATH = TRAIN_LOADER_PATH + str(k) + '.pkl'
CROSS_VALID_LOADER_PATH = VALID_LOAER_PATH + str(k) + '.pkl'
CROSS_TEST_LOADER_PATH = TEST_LOADER_PATH + str(k) + '.pkl'
trainloader = torch.load(CROSS_TRAIN_LOADER_PATH)
validloader = torch.load(CROSS_VALID_LOADER_PATH)
testloader = torch.load(CROSS_TEST_LOADER_PATH)

class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()

    # Convolutional Layers
    self.features = nn.Sequential(
      nn.Conv1d(FEATURE_SIZE, NODE_SIZE, kernel_size=KERNAL_SIZE, bias=False),
      nn.ReLU(),
      nn.Conv1d(NODE_SIZE, NODE_SIZE, kernel_size=KERNAL_SIZE, bias=False),
      nn.ReLU(),
      nn.Conv1d(NODE_SIZE, NODE_SIZE, kernel_size=KERNAL_SIZE, bias=False),
      nn.ReLU(),
      nn.Conv1d(NODE_SIZE, NODE_SIZE, kernel_size=KERNAL_SIZE, bias=False),
      nn.ReLU(),
      nn.Conv1d(NODE_SIZE, NODE_SIZE, kernel_size=KERNAL_SIZE, bias=False),
      nn.ReLU(),
    )
  
    self.fc1 = nn.Linear(NODE_SIZE*(WINDOW_SIZE-5*(KERNAL_SIZE-1)), 100)
    self.fc2 = nn.Linear(100, LABEL_SIZE)
    self.max = nn.Softmax(dim=1)

  def forward(self, x):
    x = self.features(x)
    x = x.view(x.shape[0], -1)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    x = self.max(x)
    return x

def train_save_CNN_model(TYPE, EPOCH_SIZE):
  # manually set random seed
  torch.backends.cudnn.deterministic = True
  torch.manual_seed(SEED)

  # set gpu device
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  net = CNN().double().to(device)

  # pick the criterion and optimizer
  criterion = nn.MultiLabelSoftMarginLoss()
  optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)

  print("Learning rate %.5f, batch size %d, node size %d, kernal size %d" % (LEARNING_RATE, BATCH_SIZE, NODE_SIZE, KERNAL_SIZE))

  # initialization
  train_acc_list = []
  val_acc_list = []
  test_acc_list = []
  accuray = 0

  # start to train with epoches
  for epoch in range(EPOCH_SIZE):
    running_loss = 0.0
    train_total = 0
    train_correct = 0
    valid_total = 0
    valid_correct = 0
    test_total = 0
    test_correct = 0

    # for the training dataset
    for i, data in enumerate(trainloader, 0):
      inputs, labels = data
      inputs, labels = inputs.cuda(0), labels.cuda(0)
      optimizer.zero_grad()
      outputs = net(inputs)
      train_total += labels.size(0)
      train_correct += (torch.max(outputs, 1)[1] == torch.max(labels, 1)[1]).sum().item()
      loss = criterion(outputs, labels)
      if TYPE == 'l0_norm':
        # add group lasso regularization
        lgl = 1e-8
        cnt = torch.tensor([0]).cuda(0)
        for name, param in net.named_parameters():
          if "features" in name:
            cnt = cnt + param.detach().nonzero().size(0)
            #cnt = cnt + len(param.detach()[param.detach() > 1e-2]) + len(param.detach()[param.detach() < -1e-2])
        loss = loss + lgl * cnt
      elif TYPE == 'l1_norm':
        # add group lasso regularization
        lgl = 1e-8
        regularization = torch.tensor([0]).cuda(0)
        for name, param in net.named_parameters():
          if "features" in name:
            regularization = regularization + torch.norm(param, 1)
        loss = loss + lgl * regularization
      elif TYPE == 'l2_norm':
        lgl = 1e-8
        regularization = torch.tensor([0]).cuda(0)
        for name, param in net.named_parameters():
          if "features" in name:
            regularization = regularization + torch.norm(param)
        loss = loss + lgl * regularization
      elif TYPE == 'group_lasso':
        # add group lasso regularization
        lgl = 1e-8
        regularization = torch.tensor([0]).cuda(0)
        for name, param in net.named_parameters():
          if "features" in name:
            for i in range(param.shape[0]):
              regularization = regularization + torch.norm(param[i,:,:])
        loss = loss + lgl * regularization
      elif TYPE == 'l1_group_lasso':
        lgl = 1e-8
        alpha = 0.5
        group_lasso_regularization = torch.tensor([0]).cuda(0)
        lasso_regularization = torch.tensor([0]).cuda(0)
        for name, param in net.named_parameters():
          if "features" in name:
            for i in range(param.shape[0]):
              group_lasso_regularization = group_lasso_regularization + torch.norm(param[i,:,:])
            lasso_regularization = lasso_regularization + torch.norm(param, 1)
        loss = loss + (1-alpha) * lgl * group_lasso_regularization + alpha * lgl * lasso_regularization
      elif TYPE == 'l0_group_lasso':
        #lgl = 0.000001
        #alpha = 0.90
        l0 = 1e-8
        lg = 0.4*1e-4
        cnt = torch.tensor([0]).cuda(0)
        group_lasso_regularization = torch.tensor([0]).cuda(0)
        lasso_regularization = torch.tensor([0]).cuda(0)
        for name, param in net.named_parameters():
          if "features" in name:
            for i in range(param.shape[0]):
              group_lasso_regularization = group_lasso_regularization + torch.norm(param[i,:,:])
            cnt += param.detach().nonzero().size(0)
            #cnt = cnt + len(param.detach()[param.detach() > 2*1e-2]) + len(param.detach()[param.detach() < 2*-1e-2])
            #lasso_regularization = lasso_regularization + torch.norm(param, 0)
        loss = loss + lg * group_lasso_regularization + l0 * cnt
        #loss = loss + (1-alpha) * lgl * group_lasso_regularization + alpha * lgl * lasso_regularization
      loss.backward()
      optimizer.step()
      running_loss += loss.item()

    # for the validation dataset
    for data in validloader:
      inputs, labels = data
      inputs, labels = inputs.cuda(0), labels.cuda(0)
      outputs = net(inputs)
      _, predicted = torch.max(outputs.data, 1)
      valid_total += labels.size(0)
      valid_correct += (predicted == torch.max(labels, 1)[1]).sum().item()
    
    # for the test dataset
    for data in testloader:
      inputs, labels = data
      inputs, labels = inputs.cuda(0), labels.cuda(0)
      outputs = net(inputs)
      _, predicted = torch.max(outputs.data, 1)
      test_total += labels.size(0)
      test_correct += (predicted == torch.max(labels, 1)[1]).sum().item()
    
    # obtain the results for training, validation, test dataset
    train_acc = 100 * train_correct / train_total
    valid_acc = 100 * valid_correct / valid_total
    test_acc = 100 * test_correct / test_total
    train_acc_list.append(train_acc)
    val_acc_list.append(valid_acc)
    test_acc_list.append(test_acc)
    print("epoch %d, loss %.3f, train acc %.2f%%, valid acc %.2f%%, test acc %.2f%%" % (epoch+1, running_loss, train_acc, valid_acc, test_acc))
    
    # save the best model
    if valid_acc >= accuray:
      accuray = valid_acc
      torch.save(net, PATH + '/model/' + TYPE + str(k) + ".ptl")
    
  return train_acc_list, val_acc_list, test_acc_list

## Results for CNN Model

In [ ]:
TYPE = 'no_penalty'
EPOCH_SIZE = 150
train_acc, valid_acc, test_acc = train_save_CNN_model(TYPE, EPOCH_SIZE)

In [ ]:
TYPE = 'l0_norm'
EPOCH_SIZE = 150
train_acc, valid_acc, test_acc = train_save_CNN_model(TYPE, EPOCH_SIZE)

In [ ]:
TYPE = 'l1_norm'
EPOCH_SIZE = 150
train_acc, valid_acc, test_acc = train_save_CNN_model(TYPE, EPOCH_SIZE)

In [ ]:
TYPE = 'l2_norm'
EPOCH_SIZE = 150
train_acc, valid_acc, test_acc = train_save_CNN_model(TYPE, EPOCH_SIZE)

In [ ]:
TYPE = 'group_lasso'
EPOCH_SIZE = 150
train_acc, valid_acc, test_acc = train_save_CNN_model(TYPE, EPOCH_SIZE)

In [ ]:
TYPE = 'l1_group_lasso'
EPOCH_SIZE = 150
train_acc, valid_acc, test_acc = train_save_CNN_model(TYPE, EPOCH_SIZE)

In [ ]:
TYPE = 'l0_group_lasso'
EPOCH_SIZE = 150
train_acc, valid_acc, test_acc = train_save_CNN_model(TYPE, EPOCH_SIZE)
print(train_acc)
print(valid_acc)
print(test_acc)

## Results after Pruning the above Models

In [6]:
PRUNE_THRESHOLD = 0.005

class ThresholdPruning(prune.BasePruningMethod):
    PRUNING_TYPE = "unstructured"

    def __init__(self, threshold):
        self.threshold = threshold

    def compute_mask(self, tensor, default_mask):
      return torch.abs(tensor) > self.threshold

PATHS = {'No penalty - 4 128 0.0001':            PATH + '/model/final/lr0.0001/no_penalty4.ptl',
         'l0 norm - 4 128 0.0001':               PATH + '/model/final/lr0.0001/l0_norm4.ptl',
         'l1 norm - 4 128 0.0001':               PATH + '/model/final/lr0.0001/l1_norm4.ptl',
         'l2 norm - 4 128 0.0001':               PATH + '/model/final/lr0.0001/l2_norm4.ptl',
         'group lasso - 4 128 0.0001':           PATH + '/model/final/lr0.0001/group_lasso4.ptl',
         'l1 group lasso - 4 128 0.0001':        PATH + '/model/final/lr0.0001/l1_group_lasso4.ptl',
         'l0 group lasso - 4 128 0.0001 (BEST)': PATH + '/model/final/lr0.0001/l0_group_lasso4.ptl',
         'l0 group lasso - 4 128 0.00005':       PATH + '/model/final/lr0.00005/l0_group_lasso4.ptl',
         'l0 group lasso - 4 128 0.00015':       PATH + '/model/final/lr0.00015/l0_group_lasso4.ptl',
         'l0 group lasso - 4 128 0.0002':        PATH + '/model/final/lr0.0002/l0_group_lasso4.ptl',
         'l0 group lasso - 4 128 0.00001':       PATH + '/model/final/lr0.00001/l0_group_lasso4.ptl',
         'l0 group lasso - 4 256 0.0001':        PATH + '/model/final/256node/l0_group_lasso4.ptl',
         'l0 group lasso - 4 64 0.0001':         PATH + '/model/final/64node/l0_group_lasso4.ptl',
         'l0 group lasso - 0 128 0.0001':        PATH + '/model/final/lr0.0001/l0_group_lasso0.ptl',
         'l0 group lasso - 1 128 0.0001':        PATH + '/model/final/lr0.0001/l0_group_lasso1.ptl',
         'l0 group lasso - 2 128 0.0001':        PATH + '/model/final/lr0.0001/l0_group_lasso2.ptl',
         'l0 group lasso - 3 128 0.0001':        PATH + '/model/final/lr0.0001/l0_group_lasso3.ptl',
        }

for name in PATHS:
  print('Here are the results for {}:'.format(name))
  # load the model
  net = torch.load(PATHS[name])

  # display the results before compressed model
  test_correct = 0
  test_total = 0
  with torch.no_grad():
    for data in testloader:
      inputs, labels = data
      inputs, labels = inputs.cuda(0), labels.cuda(0)
      outputs = net(inputs)
      _, predicted = torch.max(outputs.data, 1)
      test_total += labels.size(0)
      test_correct += (predicted == torch.max(labels, 1)[1]).sum().item()
    test_acc = 100 * test_correct / test_total
  print('Accuracy of the network on the %d test data: %.2f %% before compression' % (test_total, test_acc))

  # prune the model
  parameters_to_prune = []
  for na, child in net.features.named_children():
    if int(na) % 2 == 0:
      parameters_to_prune.append((child, "weight"))
  if name == 'l0 group lasso 256':
    prune.global_unstructured(parameters_to_prune, pruning_method=ThresholdPruning, threshold=PRUNE_THRESHOLD)
  else:
    prune.global_unstructured(parameters_to_prune, pruning_method=ThresholdPruning, threshold=PRUNE_THRESHOLD)

  # calculate the sparsity
  total_weight = 0
  total_nonzero = 0
  for na, child in net.features.named_children():
    if int(na) % 2 == 0:
      total_weight += torch.numel(child.weight)
      total_nonzero += torch.count_nonzero(child.weight)
  print('Nonzero Parameter for the compressed model: %.2f %%' % (100*float(total_nonzero / total_weight)))

  # display the results after compressed model
  test_correct = 0
  test_total = 0
  with torch.no_grad():
    for data in testloader:
      inputs, labels = data
      inputs, labels = inputs.cuda(0), labels.cuda(0)
      outputs = net(inputs)
      _, predicted = torch.max(outputs.data, 1)
      test_total += labels.size(0)
      test_correct += (predicted == torch.max(labels, 1)[1]).sum().item()
    test_acc = 100 * test_correct / test_total
  print('Accuracy of the network on the %d test data: %.2f %% after compression\n' % (test_total, test_acc))

Here are the results for No penalty - 4 128 0.0001:
Accuracy of the network on the 2947 test data: 91.52 % before compression
Nonzero Parameter for the compressed model: 88.49 %
Accuracy of the network on the 2947 test data: 91.48 % after compression

Here are the results for l0 norm - 4 128 0.0001:
Accuracy of the network on the 2947 test data: 91.52 % before compression
Nonzero Parameter for the compressed model: 88.49 %
Accuracy of the network on the 2947 test data: 91.48 % after compression

Here are the results for l1 norm - 4 128 0.0001:
Accuracy of the network on the 2947 test data: 90.46 % before compression
Nonzero Parameter for the compressed model: 81.58 %
Accuracy of the network on the 2947 test data: 90.33 % after compression

Here are the results for l2 norm - 4 128 0.0001:
Accuracy of the network on the 2947 test data: 91.01 % before compression
Nonzero Parameter for the compressed model: 88.35 %
Accuracy of the network on the 2947 test data: 90.94 % after compression

H

Save pruning model to android model



In [8]:
PRUNE_THRESHOLD = 0.005

class FooBarPruningMethod(prune.BasePruningMethod):
    """Prune every other entry in a tensor
    """
    PRUNING_TYPE = 'unstructured'

    def compute_mask(self, t, default_mask):
      return torch.abs(t) > PRUNE_THRESHOLD

def foobar_unstructured(module, name):
    FooBarPruningMethod.apply(module, name)
    return module

PATHS = {'l0_norm':       PATH + '/model/final/lr0.0001/l0_norm',
      'l1_norm':       PATH + '/model/final/lr0.0001/l1_norm',
      'l2_norm':       PATH + '/model/final/lr0.0001/l2_norm',
      'group_lasso':     PATH + '/model/final/lr0.0001/group_lasso',
      'l1_group_lasso':   PATH + '/model/final/lr0.0001/l1_group_lasso',
      'l0_group_lasso': PATH + '/model/final/lr0.0001/l0_group_lasso',
    }

for name in PATHS:
  # load the model
  net_uncompress = torch.load(PATHS[name] + "4.ptl")
  torch.save(net_uncompress, PATH + '/model/android_model/cuda_model/' + name + "_uncompressed.ptl")

  foobar_unstructured(net_uncompress.features[0], "weight")
  foobar_unstructured(net_uncompress.features[2], "weight")
  foobar_unstructured(net_uncompress.features[4], "weight")
  foobar_unstructured(net_uncompress.features[6], "weight")
  foobar_unstructured(net_uncompress.features[8], "weight")

  net_compress = torch.load(PATHS[name] + "4.ptl")
  net_compress.features[0].weight = torch.nn.Parameter(net_uncompress.features[0].weight)
  net_compress.features[2].weight = torch.nn.Parameter(net_uncompress.features[2].weight)
  net_compress.features[4].weight = torch.nn.Parameter(net_uncompress.features[4].weight)
  net_compress.features[6].weight = torch.nn.Parameter(net_uncompress.features[6].weight)
  net_compress.features[8].weight = torch.nn.Parameter(net_uncompress.features[8].weight)
  
  torch.save(net_compress, PATH + '/model/android_model/cuda_model/' + name + "_compressed.ptl")


Save cuda model to cpu model



In [9]:
PATHS = {'l0_norm':       PATH + '/model/android_model/cuda_model/l0_norm',
      'l1_norm':       PATH + '/model/android_model/cuda_model/l1_norm',
      'l2_norm':       PATH + '/model/android_model/cuda_model/l2_norm',
      'group_lasso':     PATH + '/model/android_model/cuda_model/group_lasso',
      'l1_group_lasso':   PATH + '/model/android_model/cuda_model/l1_group_lasso',
      'l0_group_lasso': PATH + '/model/android_model/cuda_model/l0_group_lasso',
    }

for name in PATHS:
  # load the model
  net_uncompress = torch.load(PATHS[name] + "_uncompressed.ptl", map_location='cpu')
  torch.jit.script(net_uncompress)._save_for_lite_interpreter(PATH + '/model/android_model/cpu_model/' + name + "_uncompressed.ptl")

  net_compress = torch.load(PATHS[name] + "_compressed.ptl", map_location='cpu')
  torch.jit.script(net_compress)._save_for_lite_interpreter(PATH + '/model/android_model/cpu_model/' + name + "_compressed.ptl")

Obtain the pruning results for l0 norm, l1 norm, l2 norm, group lasso, l0 group lasso and l1 group lasso



In [10]:
PATHS = {'l0_group_lasso': PATH + '/model/android_model/cuda_model/l0_group_lasso',
      'l0_norm':       PATH + '/model/android_model/cuda_model/l0_norm',
      'l1_norm':       PATH + '/model/android_model/cuda_model/l1_norm',
      'l2_norm':       PATH + '/model/android_model/cuda_model/l2_norm',
      'group_lasso':     PATH + '/model/android_model/cuda_model/group_lasso',
      'l1_group_lasso':   PATH + '/model/android_model/cuda_model/l1_group_lasso',
    }
for name in PATHS:
  print('Here are the results for {}:'.format(name))
  # load the model
  net_compress = torch.load(PATHS[name] + "_compressed.ptl", map_location='cpu')
  
  # display the results after compressed model
  test_correct = 0
  test_total = 0
  with torch.no_grad():
    for data in testloader:
      inputs, labels = data
      outputs = net_compress(inputs)
      _, predicted = torch.max(outputs.data, 1)
      test_total += labels.size(0)
      test_correct += (predicted == torch.max(labels, 1)[1]).sum().item()
    test_acc = 100 * test_correct / test_total
  print('Accuracy of the network on the %d test data: %.2f %% after compression' % (test_total, test_acc))

  total_paramaeters_uncompressed = 0
  for i in range(0,10,2):
    weight_count = 1
    for j in range(3):
      weight_count = weight_count * net_compress.features[i].weight.data.size()[j]
    total_paramaeters_uncompressed += weight_count

  new_layer_node = []
  for layer in range(0,8,2):
    node_count = 0
    if layer == 0:
      for i in range(NODE_SIZE):
        nonzero_count = False
        for j in range(FEATURE_SIZE):
          if torch.is_nonzero(torch.sum(net_compress.features[layer].weight.data[i][j])):
              nonzero_count = True
              break
        for j in range(NODE_SIZE):
          if torch.is_nonzero(torch.sum(net_compress.features[layer+2].weight.data[j][i])):
              nonzero_count = True
              break
        if nonzero_count == True:
          node_count+=1
    else:
      for i in range(NODE_SIZE):
        nonzero_count = False
        for j in range(NODE_SIZE):
          if torch.is_nonzero(torch.sum(net_compress.features[layer].weight.data[i][j])):
              nonzero_count = True
              break
        for j in range(NODE_SIZE):
          if torch.is_nonzero(torch.sum(net_compress.features[layer+2].weight.data[j][i])):
              nonzero_count = True
              break
        if nonzero_count == True:
          node_count+=1
    new_layer_node.append(node_count)
    
  new_compress_net = torch.load(PATHS[name] + "_compressed.ptl", map_location='cpu')
  index = 0
  for i in range(0,10,2):
    if i == 0:
      new_compress_net.features[i] = nn.Conv1d(FEATURE_SIZE, new_layer_node[index], kernel_size=KERNAL_SIZE, bias=False)
    elif i == 8:
      new_compress_net.features[i] = nn.Conv1d(new_layer_node[index], NODE_SIZE, kernel_size=KERNAL_SIZE, bias=False)
    else:
      new_compress_net.features[i] = nn.Conv1d(new_layer_node[index], new_layer_node[index+1], kernel_size=KERNAL_SIZE, bias=False)
      index+=1  
  node_indexes = []
  for layer in range(0,10,2):
    index = 0
    if layer == 0:
      for i in range(NODE_SIZE):
        nonzero_count = False
        for j in range(FEATURE_SIZE):
          if torch.is_nonzero(torch.sum(net_compress.features[layer].weight.data[i][j])):
              nonzero_count = True
              break
        for j in range(NODE_SIZE):
          if torch.is_nonzero(torch.sum(net_compress.features[layer+2].weight.data[j][i])):
              nonzero_count = True
              break
        if nonzero_count == True:
          for j in range(FEATURE_SIZE):
            new_compress_net.features[layer].weight.data[index][j] = net_compress.features[layer].weight.data[i][j]
          index+=1
          node_indexes.append(i)
    elif layer == 8:
      for i in range(NODE_SIZE):
        for j in range(len(node_indexes)):
          new_compress_net.features[layer].weight.data[i][j] = net_compress.features[layer].weight.data[i][node_indexes[j]]
    else:
      temp_indexes = []
      for i in range(NODE_SIZE):
        nonzero_count = False
        for j in range(NODE_SIZE):
          if torch.is_nonzero(torch.sum(net_compress.features[layer].weight.data[i][j])):
              nonzero_count = True
              break
        for j in range(NODE_SIZE):
          if torch.is_nonzero(torch.sum(net_compress.features[layer+2].weight.data[j][i])):
              nonzero_count = True
              break
        if nonzero_count == True:
          for j in range(len(node_indexes)):
            new_compress_net.features[layer].weight.data[index][j] = net_compress.features[layer].weight.data[i][node_indexes[j]]
          index+=1
          temp_indexes.append(i)
      node_indexes = temp_indexes

  # display the results after compressed model
  new_compress_net = new_compress_net.double()
  total_paramaeters_compressed = 0
  for i in range(0,10,2):
    weight_count = 1
    for j in range(3):
      weight_count = weight_count * new_compress_net.features[i].weight.data.size()[j]
    total_paramaeters_compressed += weight_count

  print('Sparity for the compressed model: %.2f %%' % (100*float(total_paramaeters_compressed) / total_paramaeters_uncompressed))

  # display node remainng
  node_count = 0
  for i in range(len(new_layer_node)): 
    node_count += new_layer_node[i]
  print('Sparity for the node remaining: %.2f %%' % (100*float(node_count) / (4.0*NODE_SIZE)))

  test_correct = 0
  test_total = 0
  with torch.no_grad():
    for data in testloader:
      inputs, labels = data
      outputs = new_compress_net(inputs)
      _, predicted = torch.max(outputs.data, 1)
      test_total += labels.size(0)
      test_correct += (predicted == torch.max(labels, 1)[1]).sum().item()
    test_acc = 100 * test_correct / test_total
  print('Accuracy of the network on the %d test data: %.2f %% after compression\n' % (test_total, test_acc))
  torch.jit.script(new_compress_net)._save_for_lite_interpreter(PATH + '/model/android_model/android_final_model/' + name + "_compressed.ptl")

Here are the results for l0_group_lasso:
Accuracy of the network on the 2947 test data: 89.96 % after compression
Sparity for the compressed model: 46.83 %
Sparity for the node remaining: 68.75 %
Accuracy of the network on the 2947 test data: 89.96 % after compression

Here are the results for l0_norm:
Accuracy of the network on the 2947 test data: 91.48 % after compression
Sparity for the compressed model: 100.00 %
Sparity for the node remaining: 100.00 %
Accuracy of the network on the 2947 test data: 91.48 % after compression

Here are the results for l1_norm:
Accuracy of the network on the 2947 test data: 90.33 % after compression
Sparity for the compressed model: 98.47 %
Sparity for the node remaining: 99.22 %
Accuracy of the network on the 2947 test data: 90.33 % after compression

Here are the results for l2_norm:
Accuracy of the network on the 2947 test data: 90.94 % after compression
Sparity for the compressed model: 100.00 %
Sparity for the node remaining: 100.00 %
Accuracy of

# Other Results -- learning rate, cross validation and threshold


k-fold cross validation results



In [11]:
PRUNE_THRESHOLD = 0.005

class FooBarPruningMethod(prune.BasePruningMethod):
    """Prune every other entry in a tensor
    """
    PRUNING_TYPE = 'unstructured'

    def compute_mask(self, t, default_mask):
      return torch.abs(t) > PRUNE_THRESHOLD

def foobar_unstructured(module, name):
    FooBarPruningMethod.apply(module, name)
    return module

PATHS = {'0':   PATH + '/model/final/lr0.0001/l0_group_lasso0.ptl',
      '1':   PATH + '/model/final/lr0.0001/l0_group_lasso1.ptl',
      '2':   PATH + '/model/final/lr0.0001/l0_group_lasso2.ptl',
      '3':   PATH + '/model/final/lr0.0001/l0_group_lasso3.ptl',
      '4':   PATH + '/model/final/lr0.0001/l0_group_lasso4.ptl',
    }

for name in PATHS:
  # load the model
  net_uncompress = torch.load(PATHS[name])
  torch.save(net_uncompress, PATH + '/model/android_model/kfold/' + name + "_uncompressed.ptl")

  foobar_unstructured(net_uncompress.features[0], "weight")
  foobar_unstructured(net_uncompress.features[2], "weight")
  foobar_unstructured(net_uncompress.features[4], "weight")
  foobar_unstructured(net_uncompress.features[6], "weight")
  foobar_unstructured(net_uncompress.features[8], "weight")

  net_compress = torch.load(PATHS[name])
  net_compress.features[0].weight = torch.nn.Parameter(net_uncompress.features[0].weight)
  net_compress.features[2].weight = torch.nn.Parameter(net_uncompress.features[2].weight)
  net_compress.features[4].weight = torch.nn.Parameter(net_uncompress.features[4].weight)
  net_compress.features[6].weight = torch.nn.Parameter(net_uncompress.features[6].weight)
  net_compress.features[8].weight = torch.nn.Parameter(net_uncompress.features[8].weight)

  torch.save(net_compress, PATH + '/model/android_model/kfold/' + name + "_compressed.ptl")

In [12]:
PATHS = {'0':  PATH + '/model/android_model/kfold/0',
      '1':  PATH + '/model/android_model/kfold/1',
      '2':  PATH + '/model/android_model/kfold/2',
      '3':  PATH + '/model/android_model/kfold/3',
      '4':  PATH + '/model/android_model/kfold/4',
    }
for name in PATHS:
  print('Here are the results for fold {}:'.format(name))
  # load the model
  net_compress = torch.load(PATHS[name] + "_compressed.ptl", map_location='cpu')
  
  # display the results after compressed model
  test_correct = 0
  test_total = 0
  with torch.no_grad():
    for data in testloader:
      inputs, labels = data
      outputs = net_compress(inputs)
      _, predicted = torch.max(outputs.data, 1)
      test_total += labels.size(0)
      test_correct += (predicted == torch.max(labels, 1)[1]).sum().item()
    test_acc = 100 * test_correct / test_total
  print('Accuracy of the network on the %d test data: %.2f %% after compression' % (test_total, test_acc))

  total_paramaeters_uncompressed = 0
  for i in range(0,10,2):
    weight_count = 1
    for j in range(3):
      weight_count = weight_count * net_compress.features[i].weight.data.size()[j]
    total_paramaeters_uncompressed += weight_count

  new_layer_node = []
  for layer in range(0,8,2):
    node_count = 0
    if layer == 0:
      for i in range(NODE_SIZE):
        nonzero_count = False
        for j in range(FEATURE_SIZE):
          if torch.is_nonzero(torch.sum(net_compress.features[layer].weight.data[i][j])):
              nonzero_count = True
              break
        for j in range(NODE_SIZE):
          if torch.is_nonzero(torch.sum(net_compress.features[layer+2].weight.data[j][i])):
              nonzero_count = True
              break
        if nonzero_count == True:
          node_count+=1
    else:
      for i in range(NODE_SIZE):
        nonzero_count = False
        for j in range(NODE_SIZE):
          if torch.is_nonzero(torch.sum(net_compress.features[layer].weight.data[i][j])):
              nonzero_count = True
              break
        for j in range(NODE_SIZE):
          if torch.is_nonzero(torch.sum(net_compress.features[layer+2].weight.data[j][i])):
              nonzero_count = True
              break
        if nonzero_count == True:
          node_count+=1
    new_layer_node.append(node_count)
    
  new_compress_net = torch.load(PATHS[name] + "_compressed.ptl", map_location='cpu')
  index = 0
  for i in range(0,10,2):
    if i == 0:
      new_compress_net.features[i] = nn.Conv1d(FEATURE_SIZE, new_layer_node[index], kernel_size=KERNAL_SIZE, bias=False)
    elif i == 8:
      new_compress_net.features[i] = nn.Conv1d(new_layer_node[index], NODE_SIZE, kernel_size=KERNAL_SIZE, bias=False)
    else:
      new_compress_net.features[i] = nn.Conv1d(new_layer_node[index], new_layer_node[index+1], kernel_size=KERNAL_SIZE, bias=False)
      index+=1  
  node_indexes = []
  for layer in range(0,10,2):
    index = 0
    if layer == 0:
      for i in range(NODE_SIZE):
        nonzero_count = False
        for j in range(FEATURE_SIZE):
          if torch.is_nonzero(torch.sum(net_compress.features[layer].weight.data[i][j])):
              nonzero_count = True
              break
        for j in range(NODE_SIZE):
          if torch.is_nonzero(torch.sum(net_compress.features[layer+2].weight.data[j][i])):
              nonzero_count = True
              break
        if nonzero_count == True:
          for j in range(FEATURE_SIZE):
            new_compress_net.features[layer].weight.data[index][j] = net_compress.features[layer].weight.data[i][j]
          index+=1
          node_indexes.append(i)
    elif layer == 8:
      for i in range(NODE_SIZE):
        for j in range(len(node_indexes)):
          new_compress_net.features[layer].weight.data[i][j] = net_compress.features[layer].weight.data[i][node_indexes[j]]
    else:
      temp_indexes = []
      for i in range(NODE_SIZE):
        nonzero_count = False
        for j in range(NODE_SIZE):
          if torch.is_nonzero(torch.sum(net_compress.features[layer].weight.data[i][j])):
              nonzero_count = True
              break
        for j in range(NODE_SIZE):
          if torch.is_nonzero(torch.sum(net_compress.features[layer+2].weight.data[j][i])):
              nonzero_count = True
              break
        if nonzero_count == True:
          for j in range(len(node_indexes)):
            new_compress_net.features[layer].weight.data[index][j] = net_compress.features[layer].weight.data[i][node_indexes[j]]
          index+=1
          temp_indexes.append(i)
      node_indexes = temp_indexes

  # display the results after compressed model
  new_compress_net = new_compress_net.double()
  total_paramaeters_compressed = 0
  for i in range(0,10,2):
    weight_count = 1
    for j in range(3):
      weight_count = weight_count * new_compress_net.features[i].weight.data.size()[j]
    total_paramaeters_compressed += weight_count

  print('Sparity for the compressed model: %.2f %%' % (100*float(total_paramaeters_compressed) / total_paramaeters_uncompressed))

  # display node remainng
  node_count = 0
  for i in range(len(new_layer_node)): 
    node_count += new_layer_node[i]
  print('Sparity for the node remaining: %.2f %%' % (100*float(node_count) / (4.0*NODE_SIZE)))

  test_correct = 0
  test_total = 0
  with torch.no_grad():
    for data in testloader:
      inputs, labels = data
      outputs = new_compress_net(inputs)
      _, predicted = torch.max(outputs.data, 1)
      test_total += labels.size(0)
      test_correct += (predicted == torch.max(labels, 1)[1]).sum().item()
    test_acc = 100 * test_correct / test_total
  print('Accuracy of the network on the %d test data: %.2f %% after compression\n' % (test_total, test_acc))
  torch.jit.script(new_compress_net)._save_for_lite_interpreter(PATH + '/model/android_model/android_final_model/' + name + "_compressed.ptl")

Here are the results for fold 0:
Accuracy of the network on the 2947 test data: 78.08 % after compression
Sparity for the compressed model: 31.99 %
Sparity for the node remaining: 56.64 %
Accuracy of the network on the 2947 test data: 78.08 % after compression

Here are the results for fold 1:
Accuracy of the network on the 2947 test data: 89.28 % after compression
Sparity for the compressed model: 64.29 %
Sparity for the node remaining: 80.27 %
Accuracy of the network on the 2947 test data: 89.28 % after compression

Here are the results for fold 2:
Accuracy of the network on the 2947 test data: 79.37 % after compression
Sparity for the compressed model: 32.25 %
Sparity for the node remaining: 56.84 %
Accuracy of the network on the 2947 test data: 79.37 % after compression

Here are the results for fold 3:
Accuracy of the network on the 2947 test data: 78.22 % after compression
Sparity for the compressed model: 40.02 %
Sparity for the node remaining: 63.48 %
Accuracy of the network on

Results for different learning rate.



In [13]:
PRUNE_THRESHOLD = 0.005

class FooBarPruningMethod(prune.BasePruningMethod):
    """Prune every other entry in a tensor
    """
    PRUNING_TYPE = 'unstructured'

    def compute_mask(self, t, default_mask):
      return torch.abs(t) > PRUNE_THRESHOLD

def foobar_unstructured(module, name):
    FooBarPruningMethod.apply(module, name)
    return module

PATHS = {'0_0001':   PATH + '/model/final/lr0.0001/l0_group_lasso4.ptl',
      '0_00015':  PATH + '/model/final/lr0.00015/l0_group_lasso4.ptl',
      '0_00001':  PATH + '/model/final/lr0.00001/l0_group_lasso4.ptl',
      '0_00005':  PATH + '/model/final/lr0.00005/l0_group_lasso4.ptl',
      '0_0002':  PATH + '/model/final/lr0.0002/l0_group_lasso4.ptl',
    }

for name in PATHS:
  # load the model
  net_uncompress = torch.load(PATHS[name])
  torch.save(net_uncompress, PATH + '/model/android_model/learning_rate/' + name + "_uncompressed.ptl")

  foobar_unstructured(net_uncompress.features[0], "weight")
  foobar_unstructured(net_uncompress.features[2], "weight")
  foobar_unstructured(net_uncompress.features[4], "weight")
  foobar_unstructured(net_uncompress.features[6], "weight")
  foobar_unstructured(net_uncompress.features[8], "weight")

  net_compress = torch.load(PATHS[name])
  net_compress.features[0].weight = torch.nn.Parameter(net_uncompress.features[0].weight)
  net_compress.features[2].weight = torch.nn.Parameter(net_uncompress.features[2].weight)
  net_compress.features[4].weight = torch.nn.Parameter(net_uncompress.features[4].weight)
  net_compress.features[6].weight = torch.nn.Parameter(net_uncompress.features[6].weight)
  net_compress.features[8].weight = torch.nn.Parameter(net_uncompress.features[8].weight)

  torch.save(net_compress, PATH + '/model/android_model/learning_rate/' + name + "_compressed.ptl")


In [14]:
PATHS = {'0_0001':       PATH + '/model/android_model/learning_rate/0_0001',
      '0_00015':       PATH + '/model/android_model/learning_rate/0_00015',
      '0_00001':        PATH + '/model/android_model/learning_rate/0_00001',
      '0_00005':     PATH + '/model/android_model/learning_rate/0_00005',
      '0_0002':    PATH + '/model/android_model/learning_rate/0_0002',
    }
for name in PATHS:
  print('Here are the results for {}:'.format(name))
  # load the model
  net_compress = torch.load(PATHS[name] + "_compressed.ptl", map_location='cpu')
  
  # display the results after compressed model
  test_correct = 0
  test_total = 0
  with torch.no_grad():
    for data in testloader:
      inputs, labels = data
      outputs = net_compress(inputs)
      _, predicted = torch.max(outputs.data, 1)
      test_total += labels.size(0)
      test_correct += (predicted == torch.max(labels, 1)[1]).sum().item()
    test_acc = 100 * test_correct / test_total
  print('Accuracy of the network on the %d test data: %.2f %% after compression' % (test_total, test_acc))

  total_paramaeters_uncompressed = 0
  for i in range(0,10,2):
    weight_count = 1
    for j in range(3):
      weight_count = weight_count * net_compress.features[i].weight.data.size()[j]
    total_paramaeters_uncompressed += weight_count

  new_layer_node = []
  for layer in range(0,8,2):
    node_count = 0
    if layer == 0:
      for i in range(NODE_SIZE):
        nonzero_count = False
        for j in range(FEATURE_SIZE):
          if torch.is_nonzero(torch.sum(net_compress.features[layer].weight.data[i][j])):
              nonzero_count = True
              break
        for j in range(NODE_SIZE):
          if torch.is_nonzero(torch.sum(net_compress.features[layer+2].weight.data[j][i])):
              nonzero_count = True
              break
        if nonzero_count == True:
          node_count+=1
    else:
      for i in range(NODE_SIZE):
        nonzero_count = False
        for j in range(NODE_SIZE):
          if torch.is_nonzero(torch.sum(net_compress.features[layer].weight.data[i][j])):
              nonzero_count = True
              break
        for j in range(NODE_SIZE):
          if torch.is_nonzero(torch.sum(net_compress.features[layer+2].weight.data[j][i])):
              nonzero_count = True
              break
        if nonzero_count == True:
          node_count+=1
    new_layer_node.append(node_count)
  new_compress_net = torch.load(PATHS[name] + "_compressed.ptl", map_location='cpu')
  index = 0
  for i in range(0,10,2):
    if i == 0:
      new_compress_net.features[i] = nn.Conv1d(FEATURE_SIZE, new_layer_node[index], kernel_size=KERNAL_SIZE, bias=False)
    elif i == 8:
      new_compress_net.features[i] = nn.Conv1d(new_layer_node[index], NODE_SIZE, kernel_size=KERNAL_SIZE, bias=False)
    else:
      new_compress_net.features[i] = nn.Conv1d(new_layer_node[index], new_layer_node[index+1], kernel_size=KERNAL_SIZE, bias=False)
      index+=1  
  node_indexes = []
  for layer in range(0,10,2):
    index = 0
    if layer == 0:
      for i in range(NODE_SIZE):
        nonzero_count = False
        for j in range(FEATURE_SIZE):
          if torch.is_nonzero(torch.sum(net_compress.features[layer].weight.data[i][j])):
              nonzero_count = True
              break
        for j in range(NODE_SIZE):
          if torch.is_nonzero(torch.sum(net_compress.features[layer+2].weight.data[j][i])):
              nonzero_count = True
              break
        if nonzero_count == True:
          for j in range(FEATURE_SIZE):
            new_compress_net.features[layer].weight.data[index][j] = net_compress.features[layer].weight.data[i][j]
          index+=1
          node_indexes.append(i)
    elif layer == 8:
      for i in range(NODE_SIZE):
        for j in range(len(node_indexes)):
          new_compress_net.features[layer].weight.data[i][j] = net_compress.features[layer].weight.data[i][node_indexes[j]]
    else:
      temp_indexes = []
      for i in range(NODE_SIZE):
        nonzero_count = False
        for j in range(NODE_SIZE):
          if torch.is_nonzero(torch.sum(net_compress.features[layer].weight.data[i][j])):
              nonzero_count = True
              break
        for j in range(NODE_SIZE):
          if torch.is_nonzero(torch.sum(net_compress.features[layer+2].weight.data[j][i])):
              nonzero_count = True
              break
        if nonzero_count == True:
          for j in range(len(node_indexes)):
            new_compress_net.features[layer].weight.data[index][j] = net_compress.features[layer].weight.data[i][node_indexes[j]]
          index+=1
          temp_indexes.append(i)
      node_indexes = temp_indexes

  # display the results after compressed model
  new_compress_net = new_compress_net.double()
  total_paramaeters_compressed = 0
  for i in range(0,10,2):
    weight_count = 1
    for j in range(3):
      weight_count = weight_count * new_compress_net.features[i].weight.data.size()[j]
    total_paramaeters_compressed += weight_count

  print('Sparity for the compressed model: %.2f %%' % (100*float(total_paramaeters_compressed) / total_paramaeters_uncompressed))

  # display node remainng
  node_count = 0
  for i in range(len(new_layer_node)): 
    node_count += new_layer_node[i]
  print('Sparity for the node remaining: %.2f %%' % (100*float(node_count) / (4.0*NODE_SIZE)))

  test_correct = 0
  test_total = 0
  with torch.no_grad():
    for data in testloader:
      inputs, labels = data
      outputs = new_compress_net(inputs)
      _, predicted = torch.max(outputs.data, 1)
      test_total += labels.size(0)
      test_correct += (predicted == torch.max(labels, 1)[1]).sum().item()
    test_acc = 100 * test_correct / test_total
  print('Accuracy of the network on the %d test data: %.2f %% after compression\n' % (test_total, test_acc))
  torch.jit.script(new_compress_net)._save_for_lite_interpreter(PATH + '/model/android_model/android_final_model/' + name + "_compressed.ptl")


Here are the results for 0_0001:
Accuracy of the network on the 2947 test data: 89.96 % after compression
Sparity for the compressed model: 46.83 %
Sparity for the node remaining: 68.75 %
Accuracy of the network on the 2947 test data: 89.96 % after compression

Here are the results for 0_00015:
Accuracy of the network on the 2947 test data: 90.91 % after compression
Sparity for the compressed model: 31.04 %
Sparity for the node remaining: 56.45 %
Accuracy of the network on the 2947 test data: 90.91 % after compression

Here are the results for 0_00001:
Accuracy of the network on the 2947 test data: 85.51 % after compression
Sparity for the compressed model: 94.66 %
Sparity for the node remaining: 97.27 %
Accuracy of the network on the 2947 test data: 85.51 % after compression

Here are the results for 0_00005:
Accuracy of the network on the 2947 test data: 89.24 % after compression
Sparity for the compressed model: 85.77 %
Sparity for the node remaining: 92.58 %
Accuracy of the network

Results for different threshold



In [16]:
PRUNE_THRESHOLD = np.arange(0.001, 0.010, 0.002)

def foobar_unstructured(module, name):
    FooBarPruningMethod.apply(module, name)
    return module

PATHS = {'l0_group_lasso':   PATH + '/model/final/lr0.0001/l0_group_lasso4.ptl',}

for threshold in PRUNE_THRESHOLD:
  class FooBarPruningMethod(prune.BasePruningMethod):
    """Prune every other entry in a tensor
    """
    PRUNING_TYPE = 'unstructured'

    def compute_mask(self, t, default_mask):
      return torch.abs(t) > threshold

  for name in PATHS:
    # load the model
    net_uncompress = torch.load(PATHS[name])
    torch.save(net_uncompress, PATH + '/model/android_model/threshold/' + str(threshold) + "_uncompressed.ptl")

    foobar_unstructured(net_uncompress.features[0], "weight")
    foobar_unstructured(net_uncompress.features[2], "weight")
    foobar_unstructured(net_uncompress.features[4], "weight")
    foobar_unstructured(net_uncompress.features[6], "weight")
    foobar_unstructured(net_uncompress.features[8], "weight")

    net_compress = torch.load(PATHS[name])
    net_compress.features[0].weight = torch.nn.Parameter(net_uncompress.features[0].weight)
    net_compress.features[2].weight = torch.nn.Parameter(net_uncompress.features[2].weight)
    net_compress.features[4].weight = torch.nn.Parameter(net_uncompress.features[4].weight)
    net_compress.features[6].weight = torch.nn.Parameter(net_uncompress.features[6].weight)
    net_compress.features[8].weight = torch.nn.Parameter(net_uncompress.features[8].weight)

    torch.save(net_compress, PATH + '/model/android_model/threshold/' + str(threshold) + "_compressed.ptl")


In [17]:
PATHS = {'0.001':       PATH + '/model/android_model/threshold/0.001',
      '0.003':       PATH + '/model/android_model/threshold/0.003',
      '0.005':        PATH + '/model/android_model/threshold/0.005',
      '0.007':     PATH + '/model/android_model/threshold/0.007',
      '0.009':    PATH + '/model/android_model/threshold/0.009',
    }
for name in PATHS:
  print('Here are the results for {}:'.format(name))
  # load the model
  net_compress = torch.load(PATHS[name] + "_compressed.ptl", map_location='cpu')
  
  # display the results after compressed model
  test_correct = 0
  test_total = 0
  with torch.no_grad():
    for data in testloader:
      inputs, labels = data
      outputs = net_compress(inputs)
      _, predicted = torch.max(outputs.data, 1)
      test_total += labels.size(0)
      test_correct += (predicted == torch.max(labels, 1)[1]).sum().item()
    test_acc = 100 * test_correct / test_total
  print('Accuracy of the network on the %d test data: %.2f %% after compression' % (test_total, test_acc))

  total_paramaeters_uncompressed = 0
  for i in range(0,10,2):
    weight_count = 1
    for j in range(3):
      weight_count = weight_count * net_compress.features[i].weight.data.size()[j]
    total_paramaeters_uncompressed += weight_count

  new_layer_node = []
  for layer in range(0,8,2):
    node_count = 0
    if layer == 0:
      for i in range(NODE_SIZE):
        nonzero_count = False
        for j in range(FEATURE_SIZE):
          if torch.is_nonzero(torch.sum(net_compress.features[layer].weight.data[i][j])):
              nonzero_count = True
              break
        for j in range(NODE_SIZE):
          if torch.is_nonzero(torch.sum(net_compress.features[layer+2].weight.data[j][i])):
              nonzero_count = True
              break
        if nonzero_count == True:
          node_count+=1
    else:
      for i in range(NODE_SIZE):
        nonzero_count = False
        for j in range(NODE_SIZE):
          if torch.is_nonzero(torch.sum(net_compress.features[layer].weight.data[i][j])):
              nonzero_count = True
              break
        for j in range(NODE_SIZE):
          if torch.is_nonzero(torch.sum(net_compress.features[layer+2].weight.data[j][i])):
              nonzero_count = True
              break
        if nonzero_count == True:
          node_count+=1
    new_layer_node.append(node_count)
    
  new_compress_net = torch.load(PATHS[name] + "_compressed.ptl", map_location='cpu')
  index = 0
  for i in range(0,10,2):
    if i == 0:
      new_compress_net.features[i] = nn.Conv1d(FEATURE_SIZE, new_layer_node[index], kernel_size=KERNAL_SIZE, bias=False)
    elif i == 8:
      new_compress_net.features[i] = nn.Conv1d(new_layer_node[index], NODE_SIZE, kernel_size=KERNAL_SIZE, bias=False)
    else:
      new_compress_net.features[i] = nn.Conv1d(new_layer_node[index], new_layer_node[index+1], kernel_size=KERNAL_SIZE, bias=False)
      index+=1  
  node_indexes = []
  for layer in range(0,10,2):
    index = 0
    if layer == 0:
      for i in range(NODE_SIZE):
        nonzero_count = False
        for j in range(FEATURE_SIZE):
          if torch.is_nonzero(torch.sum(net_compress.features[layer].weight.data[i][j])):
              nonzero_count = True
              break
        for j in range(NODE_SIZE):
          if torch.is_nonzero(torch.sum(net_compress.features[layer+2].weight.data[j][i])):
              nonzero_count = True
              break
        if nonzero_count == True:
          for j in range(FEATURE_SIZE):
            new_compress_net.features[layer].weight.data[index][j] = net_compress.features[layer].weight.data[i][j]
          index+=1
          node_indexes.append(i)
    elif layer == 8:
      for i in range(NODE_SIZE):
        for j in range(len(node_indexes)):
          new_compress_net.features[layer].weight.data[i][j] = net_compress.features[layer].weight.data[i][node_indexes[j]]
    else:
      temp_indexes = []
      for i in range(NODE_SIZE):
        nonzero_count = False
        for j in range(NODE_SIZE):
          if torch.is_nonzero(torch.sum(net_compress.features[layer].weight.data[i][j])):
              nonzero_count = True
              break
        for j in range(NODE_SIZE):
          if torch.is_nonzero(torch.sum(net_compress.features[layer+2].weight.data[j][i])):
              nonzero_count = True
              break
        if nonzero_count == True:
          for j in range(len(node_indexes)):
            new_compress_net.features[layer].weight.data[index][j] = net_compress.features[layer].weight.data[i][node_indexes[j]]
          index+=1
          temp_indexes.append(i)
      node_indexes = temp_indexes

  # display the results after compressed model
  new_compress_net = new_compress_net.double()
  total_paramaeters_compressed = 0
  for i in range(0,10,2):
    weight_count = 1
    for j in range(3):
      weight_count = weight_count * new_compress_net.features[i].weight.data.size()[j]
    total_paramaeters_compressed += weight_count

  print('Sparity for the compressed model: %.2f %%' % (100*float(total_paramaeters_compressed) / total_paramaeters_uncompressed))

  # display node remainng
  node_count = 0
  for i in range(len(new_layer_node)): 
    node_count += new_layer_node[i]
  print('Sparity for the node remaining: %.2f %%' % (100*float(node_count) / (4.0*NODE_SIZE)))

  test_correct = 0
  test_total = 0
  with torch.no_grad():
    for data in testloader:
      inputs, labels = data
      outputs = new_compress_net(inputs)
      _, predicted = torch.max(outputs.data, 1)
      test_total += labels.size(0)
      test_correct += (predicted == torch.max(labels, 1)[1]).sum().item()
    test_acc = 100 * test_correct / test_total
  print('Accuracy of the network on the %d test data: %.2f %% after compression\n' % (test_total, test_acc))
  torch.jit.script(new_compress_net)._save_for_lite_interpreter(PATH + '/model/android_model/android_final_model/' + name + "_compressed.ptl")

Here are the results for 0.001:
Accuracy of the network on the 2947 test data: 90.02 % after compression
Sparity for the compressed model: 54.68 %
Sparity for the node remaining: 74.02 %
Accuracy of the network on the 2947 test data: 90.02 % after compression

Here are the results for 0.003:
Accuracy of the network on the 2947 test data: 90.06 % after compression
Sparity for the compressed model: 48.55 %
Sparity for the node remaining: 70.12 %
Accuracy of the network on the 2947 test data: 90.06 % after compression

Here are the results for 0.005:
Accuracy of the network on the 2947 test data: 89.96 % after compression
Sparity for the compressed model: 46.83 %
Sparity for the node remaining: 68.75 %
Accuracy of the network on the 2947 test data: 89.96 % after compression

Here are the results for 0.007:
Accuracy of the network on the 2947 test data: 89.14 % after compression
Sparity for the compressed model: 44.66 %
Sparity for the node remaining: 67.38 %
Accuracy of the network on the